### QuickStart:Compare runs,choose a model and deploy it to a REST API 

In this quickstart: 
 
- Run a hyperparameter sweep on a training script  
 
- Compare the results of the runs in the mlflow uri 

- Choose the best run and register it as a model  

- Deploy the model to a REST API  

- Build a container image suitable for deployment to a cloud platform 


In [15]:
import keras  
import tensorflow as tf
import numpy as np 
import pandas as pd 
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split 

import mlflow 
from mlflow.models import infer_signature

In [16]:
##Load the Dataset
   
data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";") 
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [17]:
data["quality"].unique()

array([6, 5, 7, 8, 4, 3, 9])

In [18]:
##Split the data into training,validation and test sets 
  
train,test=train_test_split(data,test_size=0.25,random_state=25)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2009,8.0,0.40,0.33,7.70,0.034,27.0,98.0,0.99350,3.18,0.41,12.2,7
4112,6.0,0.20,0.25,2.00,0.041,30.0,95.0,0.99078,3.27,0.56,11.1,6
3598,5.8,0.20,0.24,1.40,0.033,65.0,169.0,0.99043,3.59,0.56,12.3,7
1010,7.6,0.16,0.44,1.40,0.043,25.0,109.0,0.99320,3.11,0.75,10.3,6
4344,6.7,0.27,0.69,1.20,0.176,36.0,106.0,0.99288,2.96,0.43,9.2,6
...,...,...,...,...,...,...,...,...,...,...,...,...
1175,6.4,0.25,0.41,8.60,0.042,57.0,173.0,0.99650,3.00,0.44,9.1,5
255,6.7,0.26,0.39,1.10,0.040,45.0,147.0,0.99350,3.32,0.58,9.6,8
2934,5.3,0.21,0.29,0.70,0.028,11.0,66.0,0.99215,3.30,0.40,9.8,5
2191,9.2,0.23,0.35,10.70,0.037,34.0,145.0,0.99810,3.09,0.32,9.7,5


In [19]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [20]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [21]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [22]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [23]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2025/06/14 10:17:17 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 13s 309ms/step - loss: 37.1617 - root_mean_squared_error: 6.0960
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18.4945 - root_mean_squared_error: 4.2067   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 17.4882 - root_mean_squared_error: 4.0789 - val_loss: 2.1616 - val_root_mean_squared_error: 1.4702

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 2.2883 - root_mean_squared_error: 1.5127
33/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.1132 - root_mean_squared_error: 1.4530 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.0435 - root_mean_squared_error: 1.4285 - val_loss: 1.6313 - val_root_mean_squared_error: 1.2772

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.7701 - root_mean_squared_error: 1.3305
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5337 - root_mean_squared_error: 1.2380 
46